## Import stuff

In [1]:
import csv
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import torch 
import torch.jit
import torch.nn as nn 
import torchvision
from torchvision import transforms, datasets
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.impute import KNNImputer
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

print(tf.__version__)

from tensorflow.python.client import device_lib
print("tensorflow doesn't detects GPU: ")
print(device_lib.list_local_devices() )
print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices())

print("torch detects GPU: " , torch.cuda.is_available())

oversampler = RandomOverSampler(random_state=0)
undersampler = ClusterCentroids(random_state=0)


def get_score(df_true, df_submission):
    for i in range(df_true.shape[1]):
        print("score for col ",i,": " , metrics.roc_auc_score(df_true[:,i], df_submission[:,i]))


ModuleNotFoundError: No module named 'tensorflow'

## Load training data from csv


In [ ]:
features_df_normalized = pd.read_csv("train_features_normalized.csv")
labels_df = pd.read_csv("train_labels.csv")

X = np.array(features_df_normalized.iloc[:, 2:].values,  dtype=float).reshape(227940//12,35*12)


X_train = X[:int(X.shape[0]*0.8)]
X_valid = X[int(X.shape[0]*0.8):]


Y = np.array(labels_df.iloc[:, 1:].values,  dtype=float)
Y_train = Y[:int(Y.shape[0]*0.8)]
Y_valid = Y[int(Y.shape[0]*0.8):]


print("X shape: ", X.shape)
print("Y shape: ", Y.shape)


X shape:  (18995, 420)
Y shape:  (18995, 15)


## The function that trains and rates models

In [ ]:
def find_best_model(col, X_train, Y_train, X_valid, Y_valid, X_test):
    models = []

    #task 1 and 2
    if(col<11):
        print("training sigmoid nn for col ", col)
        nn2 = tf.keras.Sequential([
            layers.Dense(64, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='sigmoid')
        ])
        nn2.compile(optimizer='adam', loss='binary_crossentropy')
        nn2.fit(X_train, Y_train[:,col], epochs=15, verbose=False)
        models.append(nn2)

        print("training lr for col ", col)
        lr = LogisticRegression(max_iter=1000)
        lr.fit(X_train, Y_train[:,col])
        models.append(lr)

        print("training rfc for col ", col)
        rfc = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=10)
        rfc.fit(X_train, Y_train[:,col])
        models.append(rfc)

        print("training adaboost for col ", col)
        adaboost = AdaBoostClassifier(n_estimators=100, random_state=10)
        adaboost.fit(X_train, Y_train[:,col])
        models.append(adaboost)
    
    #task 3
    if(col>=11):
        print("training 100+25 relu for col ", col)
        nn = tf.keras.Sequential([
        tf.keras.layers.Dense(units=100, input_shape=[420], activation='relu'),
        tf.keras.layers.Dense(units=25, activation='relu'),
        tf.keras.layers.Dense(units=1)
        ])
        nn.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.1), metrics=['mean_squared_error'])
        nn.fit(X_train, Y_train[:,col], epochs=30, verbose=False)
        models.append(nn)

        print("training 64+64 relu for col ", col)
        nn = tf.keras.Sequential([
        tf.keras.layers.Dense(units=64, input_shape=[420], activation='relu'),
        tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=1)
        ])
        nn.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.1), metrics=['mean_squared_error'])
        nn.fit(X_train, Y_train[:,col], epochs=30, verbose=False)
        models.append(nn)

    
    print("done training all models for col ", col)

    best_score = 0
    best_model = None
    for model in models:
        pred = model.predict(X_valid)
        if(col>=11):
            score = 0.5 + 0.5 * np.maximum(0, metrics.r2_score(Y_valid[:,col], pred))
        else:
            score = metrics.roc_auc_score(Y_valid[:,col], pred)
        print("score for",type(model),": " , score)
        if score > best_score:
            best_score = score
            best_model = model

    print("best model: ", type(best_model))
    predictions = best_model.predict(X_test)
    print("predictions shape: ", predictions.shape)

    return predictions

## Writing results

In [ ]:

X_test = pd.read_csv("test_features_normalized.csv")
X_test = np.array(X_test.iloc[:, 2:].values,  dtype=float).reshape(151968//12,35*12)

for i in range(11,15):
    pred = find_best_model(i, X_train, Y_train, X_valid, Y_valid, X_test)

# print(pred)


# pred1 = []
# for i in range(0, 15):
#     Y_pred_lr = pred1.append(models[i].predict_proba(test_X_normalized)[:,1])


# pred1 = np.array(pred1).transpose()
# # pred1 = model1.predict(test_X_normalized)
# pred2 = model2.predict(test_X_normalized)
# # pred2 = model2.predict(test_X_small)
# # pred2 = (pred2+1)/2
# # np.clip(pred2, 0, 1, out=pred2)
# pred3 = model3.predict(test_X_normalized)

# print(pred2[pred2==0].shape)
# print(pred2[pred2>0.5].shape)


training 100+25 relu for col  11
training 64+64 relu for col  11
training 100+25 sigmoid for col  11
training 64+64 sigmoid for col  11
done training all models for col  11
score for <class 'tensorflow.python.keras.engine.sequential.Sequential'> :  0.66358745535248
score for <class 'tensorflow.python.keras.engine.sequential.Sequential'> :  0.6819195674064146
score for <class 'tensorflow.python.keras.engine.sequential.Sequential'> :  0.5
score for <class 'tensorflow.python.keras.engine.sequential.Sequential'> :  0.5
best model:  <class 'tensorflow.python.keras.engine.sequential.Sequential'>
predictions shape:  (12664, 1)
training 100+25 relu for col  12
training 64+64 relu for col  12
training 100+25 sigmoid for col  12
training 64+64 sigmoid for col  12
done training all models for col  12
score for <class 'tensorflow.python.keras.engine.sequential.Sequential'> :  0.7952134257109678
score for <class 'tensorflow.python.keras.engine.sequential.Sequential'> :  0.8024785438972719
score for

In [ ]:
sample = pd.read_csv("sample.csv")
# print(sample.shape)
submit =  pd.DataFrame(np.zeros((12664,16)), columns=[sample.columns])
# print(submit.shape)
# print(pred1.shape)
# print(pred2.shape)
# print(pred3.shape)
# # print(submit)


 submit.iloc[:,0] = sample.iloc[:,0]
 submit.iloc[:,1:11] = pred1
 submit.iloc[:,11] = pred2
 submit.iloc[:,12:] = pred3

 submit.to_csv('submit.zip', index=False, float_format='%.3f', compression='zip')
# submit.to_csv('submit.csv', index=False, float_format='%.3f')

(12664, 16)
(12664, 16)
(12664, 10)
(12664,)
(12664, 4)


new task

In [ ]:
import csv
data = pd.read_csv("nba.csv")
   
list(data.columns.values.tolist())  
with open('train_features.csv') as train:
      

    reader = csv.reader(train)
      
    # Iterate over each column in the csv 
    # file using reader object
    for column in reader:
        col_list = df.column.values.tolist()
        ratio = col_list[0]//col_list[-1]
    # Iterate over each patient
        for row in reader:
            row.append(ratio)


new task

In [ ]:
for k in range(35):
 df = pd.read_csv('train_features_normalized_withnancols.csv', delimiter=',')
 col_list = df.column.values.tolist() 
 new_column = df[col_list[-1]] + 1
 df['ratio'] = new_column #renaming still needed
 

In [ ]:
df = pd.read_csv('train_features_normalized_withnancols.csv', delimiter=',')
col_list = df.column.values.tolist() 
for k in range(35):
 col = col_list[k]
 df.col.tolist()
 ratio = col[0]//col[-1]
 for row in df:
     row.append(ratio)

